In [1]:
import beta_glu
import optimization
import pandas as pd
import numpy as np
import warnings
from sklearn import preprocessing
import pandascharm as pc
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pickle

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [7]:
df = pd.read_excel('../beta-glucosidase/data/KcatKm w_ Temperatures.xlsx')

In [8]:
# trigram
output = 'pNP-Glc kcat/Km (1/smM)'
encoding = 'trigram'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [9]:
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [10]:
# XGB

param_grid = {'colsample_bytree': [1.0], 'gamma': [0.5], 'max_depth': [5], 'min_child_weight': [1], 'subsample': [0.6]}
XGB_trigram=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
XGB_trigram.fit(X, y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n...e,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                               

In [11]:
filename = 'XGB_trigram_12removal.sav'
pickle.dump(XGB_trigram, open(filename, 'wb'))

In [6]:
# Random Forest 
param_grid={'max_depth': [70], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [50]}
RF_trigram=GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
RF_trigram.fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [70], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [50]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [7]:
# Quadrogram
encoding = 'quadrogram'
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [8]:
param_grid={'max_depth': [70], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [50]}
RF_quadrogram=GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
RF_quadrogram.fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [70], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [50]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [19]:
filename = 'RF_quadrogram.sav'
pickle.dump(RF_quadrogram, open(filename, 'wb'))

In [9]:
# checking

inp = 'MANFPKGFLFGTATSSYQIEGAVNEDGRTPSIWDTFSKTSGMTYNGDTGDIACDHYHRYKEDVVILKEIGVKAYRFSIAWPRIFPEKGNFNPKGIDFYKRLVEELLKNDIIPVATIYHWDLPQWAGDLGGWLNRDLIYWYSEYSQKLFKEIGNVVPMWITHNEPWCASILSYGIGEHAPGHKDYREALIAAHHILLSHGEAVKIFRDMNIKESQIGITLNLTPAYPASERDVDRLAAQYADGFSNRWFLDPIFKGNYPEDMIELYKEEIGKFDFIKSEDLGIISQPIDFLGINFYSRSIVKYSEKSMLKWIGVEGPGAKTDMGWEIRPESLYDLLKRLDKEYTRIPIYITENGAAFKDIITEDGKVHDQERIEYIKEHLKYANKFIKEGGNLKGYFLWSFLDNFEWAFGYSKRFGIVYVDYKTQKRILKDSALWYKEVINRASIVF'
temper = '60'

In [10]:
encoding = 'trigram'
before, new_input =  optimization.encode_input(inp, encoding, df,output, key = None, aln = aln, temper=temper)

In [11]:
new_input

array([[-3.75948537e-17, -6.26580896e-17,  1.00000000e+00, ...,
         0.00000000e+00, -3.75948537e-17,  6.00000000e+01]])

In [12]:
y= XGB_trigram.predict(new_input)

In [13]:
y

array([45.582478], dtype=float32)

In [23]:
# checking to see the optimization process
inp = 'MANFPKGFLFGTATSSYQIEGAVNEDGRTPSIWDTFSKTSGMTYNGDTGDIACDHYHRYKEDVVILKEIGVKAYRFSIAWPRIFPEKGNFNPKGIDFYKRLVEELLKNDIIPVATIYHWDLPQWAGDLGGWLNRDLIYWYSEYSQKLFKEIGNVVPMWITHNEPWCASILSYGIGEHAPGHKDYREALIAAHHILLSHGEAVKIFRDMNIKESQIGITLNLTPAYPASERDVDRLAAQYADGFSNRWFLDPIFKGNYPEDMIELYKEEIGKFDFIKSEDLGIISQPIDFLGINFYSRSIVKYSEKSMLKWIGVEGPGAKTDMGWEIRPESLYDLLKRLDKEYTRIPIYITENGAAFKDIITEDGKVHDQERIEYIKEHLKYANKFIKEGGNLKGYFLWSFLDNFEWAFGYSKRFGIVYVDYKTQKRILKDSALWYKEVINRASIVF'
temperature = '60'
model = XGB_trigram
TaBGL = [17, 117,161,162,294,350,404,405]
PfBGL = [17, 150, 206,207,372,410,417,418]

In [24]:
max_output, final_input = optimization.stochastic_optimizer(inp, encoding, df, output, model, key = None, aln = aln, temper=temperature, epoch=10, number_of_rand =3, active_site = TaBGL )

In [25]:
final_input

'MANFPKGFLFGTATSSYVIEGAVNEDGRTPSIWDTFSKTSGMTYNGDTGDIACDHYHRYKEDVVILKEIGVKAYRFSIAWPRIFPEKGNFNPKGIDFYKRLVEELLKNDIIPVATIYQWDLPQWAGDLGGWLNRDLIYWYSEYSQKLFKEIGNVVPMWITHCIPWCASILSYGIGEHAPGHKDYREALIAAHHILLSHGEAVKIFRDMNIKESQIGITLNLTPAYPASERDVDRLAAQYADGFSNRWFLDPIFKGNYPEDMIELYKEEIGKFDFIKSEDLGIISQPIDFLGINFNSRSIVKYSEKSMLKWIGVEGPGAKTDMGWEIRPESLYDLLKRLDKEYTRIPIYITDNGAAFKDIITEDGKVHDQERIEYIKEHLKYANKFIKEGGNLKGYFLWSFLDNFAYAFGYSKRFGIVYVDYKTQKRILKDSALWYKEVINRASIVF'

In [26]:
max_output

array([45.582478], dtype=float32)

In [20]:
final_input = 'MANFPKGFLFGTATSSYQIEGAVNERGRTPSIWDTFSKTSGMTYNGDTGDIACDHYHRYKEDVVILKEIGVKAYRFSIAWPRIFPEKGNFNPKGIDFYKRLVEELLKNDIIPVATIYHWDLPQWAGDLGGWLNRDLIYWYSEYSQKLFKEIGNVVPMWITHNEPWCASILSYGIGEHAPGHKDYREALIAAHHILLSHGEAVKIFRDMNIKESQIGITLNLTPAYPASERDVDRLAAQYADGFSNRWFLDPIFKGNYPEDMIELYKEEIGKFDFIWSNDLGIISQPIDFLGINFYSRSIVKYSEKSMLKWIGVEGPGAKTDMGWEIRPESLYDLLKRLDKEYTRIPIYITENGAAFKDIITEDGKVHDQERIEYIKEHLKYANKFIKEGGNLKGYFLWSFLDNFEWAFGYSKRFGIVYVDYKTQKRILKDSALWYKEVINRASIVF'
check = optimization.spotter(inp,final_input)

In [21]:
check

['D26R', 'K276W', 'E278N']

In [6]:
# checking to see the optimization process
inp = 'MANFPKGFLFGTATSSYQIEGAVNEDGRTPSIWDTFSKTSGMTYNGDTGDIACDHYHRYKEDVVILKEIGVKAYRFSIAWPRIFPEKGNFNPKGIDFYKRLVEELLKNDIIPVATIYHWDLPQWAGDLGGWLNRDLIYWYSEYSQKLFKEIGNVVPMWITHNEPWCASILSYGIGEHAPGHKDYREALIAAHHILLSHGEAVKIFRDMNIKESQIGITLNLTPAYPASERDVDRLAAQYADGFSNRWFLDPIFKGNYPEDMIELYKEEIGKFDFIKSEDLGIISQPIDFLGINFYSRSIVKYSEKSMLKWIGVEGPGAKTDMGWEIRPESLYDLLKRLDKEYTRIPIYITENGAAFKDIITEDGKVHDQERIEYIKEHLKYANKFIKEGGNLKGYFLWSFLDNFEWAFGYSKRFGIVYVDYKTQKRILKDSALWYKEVINRASIVF'
temperature = '60'
number_of_rand = 4

In [8]:
TaBGL = [17, 117,161,162,294,350,404,405]
PfBGL = [17, 150, 206,207,372,410,417,418]